In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from netam import framework, models

In [4]:
train_df, val_df = framework.load_shmoof_dataframes("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv")# , sample_count=5000)

In [5]:
# Tang data

# full_df = pd.read_csv("/Users/matsen/data/tang-deepshm_size2_edges_22-May-2023.branch_length.csv", index_col=0).reset_index(drop=True)
# 
# # only keep rows where parent is different than child
# full_df = full_df[full_df["parent"] != full_df["child"]]
# 
# train_df = full_df.sample(frac=0.8)
# val_df = full_df.drop(train_df.index)


In [6]:
kmer_length = 5
max_length = 410

train_dataset = framework.SHMoofDataset(train_df, kmer_length=kmer_length, max_length=max_length)
val_dataset = framework.SHMoofDataset(val_df, kmer_length=kmer_length, max_length=max_length)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

we have 35830 training examples and 13186 validation examples


In [7]:
train_df["parent"].str.len().median()
#train_df["v_int_end"].hist(bins=100)
# get the index of the maximum True entry of train_dataset[0][1]
masks = train_dataset.masks
masks.shape

torch.Size([35830, 410])

In [8]:
def sequence_length_of_masks(masks):
    batch_size, seq_length = masks.shape

    range_tensor = torch.arange(seq_length).repeat(batch_size, 1)
    masked_range = torch.where(masks, range_tensor, torch.tensor(-1))

    return masked_range.max(dim=1)[0]

sequence_length_of_masks(masks).median()

tensor(364)

In [9]:
class TwoLengthSHMoofModel(nn.Module):
    def __init__(self, dataset, length_threshold, residual_penalty_weight=0.0):
        super(TwoLengthSHMoofModel, self).__init__()
        self.kmer_count = len(dataset.kmer_to_index)
        self.site_count = dataset.max_length
        self.length_threshold = length_threshold
        self.residual_penalty_weight = residual_penalty_weight

        self.kmer_embedding = nn.Embedding(self.kmer_count, 1)
        self.log_site_rates = nn.Embedding(self.site_count, 1)
        self.log_site_rates_long_residual = nn.Embedding(self.site_count, 1)

    def forward(self, encoded_parents, masks):
        log_kmer_rates = self.kmer_embedding(encoded_parents).squeeze()
        sequence_length = sequence_length_of_masks(masks)

        # Determine if the sequence is long or short
        is_long = sequence_length > self.length_threshold
        is_long = is_long.unsqueeze(-1)

        log_site_rates_short = self.log_site_rates.weight.T.expand_as(log_kmer_rates)
        # set log_site_rates_short to zero for all indices beyond the length threshold
        log_site_rates_short = torch.where(is_long, torch.tensor(0.0), log_site_rates_short)
        log_site_rates_long_residual = self.log_site_rates_long_residual.weight.T.expand_as(log_kmer_rates)
        
        # Adjust log_site_rates for long sequences
        log_site_rates_long = log_site_rates_short + log_site_rates_long_residual
        log_site_rates = torch.where(is_long, log_site_rates_long, log_site_rates_short)

        rates = torch.exp(log_kmer_rates + log_site_rates)
        return rates

    def regularization_loss(self):
        # Calculate L2 norm (squared sum) of the log_site_rates_long_residual weights
        reg_loss = torch.sum(self.log_site_rates_long_residual.weight ** 2)
        # Apply the regularization weight
        reg_loss *= self.residual_penalty_weight
        return reg_loss


In [12]:
length_threshold = sequence_length_of_masks(masks).median()
model = TwoLengthSHMoofModel(train_dataset, length_threshold=length_threshold, residual_penalty_weight=1e-5)
burrito = framework.Burrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.1, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=100)

starting training...
Epoch [1/100]	 Loss: 0.066157887	 Val Loss: 0.067732337
Epoch [2/100]	 Loss: 0.059485697	 Val Loss: 0.067316885
Epoch [3/100]	 Loss: 0.059339037	 Val Loss: 0.067260014
Epoch [4/100]	 Loss: 0.059325151	 Val Loss: 0.067260171
Epoch [5/100]	 Loss: 0.059315414	 Val Loss: 0.067251524
Epoch [6/100]	 Loss: 0.059302538	 Val Loss: 0.067225636
Epoch [7/100]	 Loss: 0.059294048	 Val Loss: 0.067249244
Epoch [8/100]	 Loss: 0.059288405	 Val Loss: 0.067247988
Epoch [9/100]	 Loss: 0.059276572	 Val Loss: 0.067150525
Epoch [10/100]	 Loss: 0.059265354	 Val Loss: 0.067193323
Epoch [11/100]	 Loss: 0.059248385	 Val Loss: 0.067125562
Epoch [12/100]	 Loss: 0.059231946	 Val Loss: 0.067146557
Epoch [13/100]	 Loss: 0.059227465	 Val Loss: 0.067140751
Epoch [14/100]	 Loss: 0.059211675	 Val Loss: 0.067143321
Epoch [15/100]	 Loss: 0.059214812	 Val Loss: 0.067136307
Epoch [16/100]	 Loss: 0.059189886	 Val Loss: 0.067144968
Epoch 00016: reducing learning rate of group 0 to 2.0000e-02.
Epoch [17/100]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load motif mutabilities
shmoof_motifs = pd.read_csv('_ignore/original_shmoof/mutabilities_context.tsv', sep='\t')
# rename Mutability column to Mutability_shmoof
#shmoof_motifs = shmoof_motifs.rename(columns={'Mutability': 'Mutability_shmoof'})
reshmoof_mutabilities = torch.exp(model.kmer_embedding.weight).squeeze().detach().numpy()
reshmoof_motifs = pd.DataFrame({'Mutability': reshmoof_mutabilities, 'Motif': train_dataset.kmer_to_index.keys()})

# Merge dataframes
merged_motifs = pd.merge(shmoof_motifs, reshmoof_motifs, on='Motif', how='inner', suffixes=('_shmoof', '_reshmoof'))

# Scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(merged_motifs['Mutability_shmoof'], merged_motifs['Mutability_reshmoof'], alpha=0.5)

# Determine bounds for y=x line
min_bound = min(merged_motifs['Mutability_shmoof'].min(), merged_motifs['Mutability_reshmoof'].min())
max_bound = max(merged_motifs['Mutability_shmoof'].max(), merged_motifs['Mutability_reshmoof'].max())

# Add y=x line
plt.plot([min_bound, max_bound], [min_bound, max_bound], 'r--')

plt.xlabel('Shmoof Mutability')
plt.ylabel('Reshmoof Mutability')
plt.title('Comparison of Motif Mutabilities: Shmoof vs. Reshmoof')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import torch
import pandas as pd

# Assuming `model` is your trained model instance
log_site_rates_short = model.log_site_rates.weight.data.squeeze().cpu().numpy()
log_site_rates_long_residual = model.log_site_rates_long_residual.weight.data.squeeze().cpu().numpy()

# Calculate the exponential of the log rates
site_rates_short = torch.exp(torch.tensor(log_site_rates_short)).numpy()
site_rates_long_residual = torch.exp(torch.tensor(log_site_rates_long_residual)).numpy()

# Ensure the site rates are 1-dimensional
assert site_rates_short.ndim == 1
assert site_rates_long_residual.ndim == 1

# Create a DataFrame for plotting
df = pd.DataFrame({
    'Position': range(1, len(site_rates_short) + 1),
    'Short_Site_Rates': site_rates_short,
    'Long_Residual_Site_Rates': site_rates_long_residual
})

# Line plot
plt.figure(figsize=(10, 6))
plt.plot(df['Position'], df['Short_Site_Rates'], label='Short Site Rates', color='orange')
plt.plot(df['Position'], df['Long_Residual_Site_Rates'], label='Long Site Residuals', color='green', alpha=0.5)
plt.xlabel('Position')
plt.ylabel('Mutability')
plt.title(f'Comparison of Short vs. Long Site Rates, residual_penalty_weight={model.residual_penalty_weight}')
plt.legend()
plt.grid(True)
plt.show()
